In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

import nbimporter
import PaneResistance as PR
import ClosedCavityResistance as CCR
import SurfaceResistance as SR

Importing Jupyter notebook from PaneResistance.ipynb
Importing Jupyter notebook from ClosedCavityResistance.ipynb
Importing Jupyter notebook from SurfaceResistance.ipynb


# グレージング複合体の熱平衡を計算するモジュール

## 1. Functions  

グレージング複合体の熱平衡モデルによって面材間の表面温度と熱抵抗を計算する。

### 入力値  

$ d_{gl,j,l} $ ：層$j$の面材を構成する材料$l$の厚さ (m)  
$ \lambda_{gl,j,l} $ ：層$j$の面材を構成する材料$l$の熱伝導率 [W/(m･K)]  
$ I_{\alpha,j} $ ：層$j$の面材の吸収日射量 (W/m<sup>2</sup>)  
$ \epsilon_{gl,f} $ ：層$j$の面材の正面側の放射率  
$ \epsilon_{gl,b} $ ：層$j$の面材の背面側の放射率  
$ d_{gap,j} $ ：層$j-1$と層$j$の面材間の中空層の厚さ (m)  
gas1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gas2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gasratio1：気体1の容積割合（0.0～1.0）  
gasdir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）  
season：季節のflag（夏期：0、冬期：1）  
$ \theta_{ex} $ ：外気温 (℃)  
$ \theta_{in} $ ：室温 (℃)  

### 出力値
$ \theta_{f,j} $ ：層$j$の面材の正面側表面温度 (℃)  
$ \theta_{b,j} $ ：層$j$の面材の背面側表面温度 (℃)  
$ R_{s,j-1,j} $ ：層$j-1$と層$j$の面材間の中空層の熱抵抗 (m<sup>2</sup>･K/W)  
$ R_{sur, ex} $ ：屋外側表面熱伝達抵抗 (m<sup>2</sup>･K/W)  
$ R_{sur, in} $ ：室内側表面熱伝達抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ d_{gl} $・$ \lambda_{gl} $・$ I_{\alpha} $・$ \epsilon_{gl,f} $・$ \epsilon_{gl,b} $の組み合わせをグレージング層の物性値のクラス『glp＿input』と定義する。  
$ d_{gap} $・gas1・gas2・gasratio1・gasdirの組み合わせをグレージング層間の中空層の物性値のクラス『gpp＿input』と定義する。   
season・$ \theta_{ex} $・$ \theta_{in} $の組み合わせを気象条件のクラス『wp＿input』と定義する。  


In [2]:
class glp_input():
    def __init__( self, d, lmd, ia, epf, epb ):
        self.d = d
        self.lmd = lmd
        self.ia = ia
        self.epf = epf
        self.epb = epb
        
#d：グレージング層jの厚さ(m)
#lmd：グレージング層jの熱伝導率[W/(m・K)]
#ia：グレージング層jの吸収日射量(W/m2)
#epf：グレージング層jの正面側の放射率
#epb：グレージング層jの背面側の放射率

class gpp_input():
    def __init__( self, d, gas1, gas2, gas1ratio, gasdir ):
        self.d = d
        self.gas1 = gas1
        self.gas2 = gas2
        self.gas1ratio = gas1ratio
        self.gasdir = gasdir
        
#d：グレージング層間の中空層の厚さ(m)
#gas1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
#gas2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
#gas1ratio：気体1の容積割合（0.0～1.0）
#gasdir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）

class wp_input():
    def __init__( self, season, te, ti ):
        self.season = season
        self.te = te
        self.ti = ti

#season：季節のflag（夏期：0、冬期：1）
#te：外気温(℃)
#ti：室温（℃）

#### 表面温度と熱抵抗の収束計算  

各面材間における日射吸収と熱伝達との熱平衡の計算方法を以下に示す。  

（参考：JIS A 2103：2014 5.4.4 面材間の中空層の熱抵抗）  

$$
\begin{eqnarray}
 &\displaystyle \frac{I_{\alpha,j}}{2} = \frac{\theta_{f,j} - \theta_{b,j-1}}{R_{s,j}(\theta_{b,j-1}, \theta_{f,j})} + \frac{\theta_{f,j} - \theta_{b,j}}{R_{gl,j}}& \qquad\qquad\text{(1)} \nonumber\\
 \nonumber \\
 &\displaystyle \frac{I_{\alpha,j}}{2} = \frac{\theta_{b,j} - \theta_{f,j}}{R_{gl,j}} + \frac{\theta_{b,j} - \theta_{f,j+1}}{R_{s,j+1}(\theta_{b,j}, \theta_{f,j+1})}& \qquad\qquad\text{(2)} \nonumber\\
\end{eqnarray}
$$

ここで、  

$ I_{\alpha,j} $ ：層$j$での吸収日射量 (W/m<sup>2</sup>)  
$ \theta_{f,j} $ ：層$j$の面材の正面側表面温度 (℃)  
$ \theta_{b,j} $ ：層$j$の面材の背面側表面温度 (℃)  
$ R_{gl,j} $ ：層$j$の面材の熱抵抗 (m<sup>2</sup>･K/W)  
$ R_{s,j}(\theta_{b,j-1}, \theta_{f,j}) $ ：層$j-1$と層$j$の面材間の中空層の熱抵抗 (m<sup>2</sup>･K/W) 

である。 

$ \theta_{b,-1} = \theta_{ex} $、$ \theta_{f,n} = \theta_{in} $、$ R_{s, 0} = R_{sur, ex} $、$ R_{s, n} = R_{sur, in} $として、式(1)と式(2)から得られる$n$層のグレージング複合体の熱平衡の連立方程式を以下に示す。  

$$ 
\begin{pmatrix} 
 \frac{1}{R_{sur, ex}}+\frac{1}{R_{gl, 0}} &-\frac{1}{R_{gl, 0}} &&&&& \\
 -\frac{1}{R_{gl, 0}} &\frac{1}{R_{gl, 0}}+\frac{1}{R_{s, 1}} &-\frac{1}{R_{s, 1}} &&&& \\
 &-\frac{1}{R_{s, 1}} &\frac{1}{R_{s, 1}}+\frac{1}{R_{gl, 1}} &-\frac{1}{R_{gl, 1}} &&O& \\
 &&-\frac{1}{R_{gl, 1}} &\frac{1}{R_{gl, 1}}+\frac{1}{R_{s, 2}} &&& \\
 &&&&&\ddots&& \\
 &&&&\ddots&&& \\
 &O&&\ddots&&&& \\
 &&&&&\frac{1}{R_{s, n-1}}+\frac{1}{R_{gl, n-1}} &-\frac{1}{R_{gl, n-1}}  \\
 &&&&&-\frac{1}{R_{gl, n-1}} &\frac{1}{R_{gl, n-1}}+\frac{1}{R_{sur, in}}  \\
\end{pmatrix} 
\begin{pmatrix} 
 \theta_{f, 0} \\
 \theta_{b, 0} \\
 \theta_{f, 1} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \theta_{b, n-1} \\
\end{pmatrix} 
= 
\begin{pmatrix} 
 \frac{I_{\alpha, 0}}{2} + \frac{\theta_{ex}}{R_{sur, ex}} \\ 
 \frac{I_{\alpha, 0}}{2} \\
 \frac{I_{\alpha, 1}}{2} \\ 
 \frac{I_{\alpha, 1}}{2} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \frac{I_{\alpha, n-1}}{2} \\ 
 \frac{I_{\alpha, n-1}}{2} + \frac{\theta_{in}}{R_{sur, in}} \\
\end{pmatrix} 
\qquad\qquad\text{(3)}
$$

![image1.png][attached1]
[attached1]:img\image1.png  

<div style="text-align: center;">
$n$層のグレージング複合体の熱平衡モデル
</div>

ここで、$ R_{gl,j} $ = $ R_{2j+1} $、$ R_{s,j}(\theta_{b,j-1}, \theta_{f,j}) $ = $ R_{2j} $、$ \theta_{f,j} $ = $ \theta_{2j+1} $、$ \theta_{b,j} $ = $ \theta_{2(j+1)} $、$ \frac{I_{\alpha,j}}{2} = q_{\alpha,2j+1} $、$ \frac{I_{\alpha,j}}{2} = q_{\alpha,2(j+1)} $と置き換えた場合の面材と中空層の層の番号の関係を以下に示す。

$$ 
\begin{pmatrix} 
 \frac{1}{R_{0}}+\frac{1}{R_{1}} & -\frac{1}{R_{1}} &&&&& \\
 -\frac{1}{R_{1}} & \frac{1}{R_{1}}+\frac{1}{R_{2}} & -\frac{1}{R_{2}} &&&& \\
 &-\frac{1}{R_{2}} & \frac{1}{R_{2}}+\frac{1}{R_{3}} & -\frac{1}{R_{3}} &&O& \\
 &&-\frac{1}{R_{3}} & \frac{1}{R_{3}}+\frac{1}{R_{4}} &&& \\
 &&&&&\ddots&& \\
 &&&&\ddots&&& \\
 &O&&\ddots&&&& \\
 &&&&& \frac{1}{R_{2(n-1)}}+\frac{1}{R_{2(n-1)+1}} & -\frac{1}{R_{2(n-1)+1}}  \\
 &&&&&-\frac{1}{R_{2(n-1)+1}} & \frac{1}{R_{2(n-1)+1}}+\frac{1}{R_{2n}}  \\
\end{pmatrix} 
\begin{pmatrix} 
 \theta_{1} \\
 \theta_{2} \\
 \theta_{3} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \theta_{2n} \\
\end{pmatrix} 
= 
\begin{pmatrix} 
 \frac{q_{\alpha,1}}{2} + \frac{\theta_{0}}{R_{0}} \\ 
 \frac{q_{\alpha,2}}{2} \\
 \frac{q_{\alpha,3}}{2} \\ 
 \frac{q_{\alpha,4}}{2} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \frac{q_{\alpha,2(n-1)+1}}{2} \\ 
 \frac{q_{\alpha,2n}}{2} + \frac{\theta_{2n+1}}{R_{2n}} \\
\end{pmatrix} 
\qquad\qquad\text{(4)}
$$

![image2.png][attached2]
[attached2]:img\image2.png

<div style="text-align: center;">
$n$層のグレージング複合体の熱平衡モデル（記号置換後）
</div>

式(4)の左辺の熱抵抗$R$に関する行列は正定値対称行列であるため、修正コレスキー分解によって熱抵抗$R$及び温度$\theta$を求める。

In [3]:
def Heat_balance(L, M, N):
    
    #面材の層数
    n = len(L.d)
    
    #面材の熱抵抗
    Rgl = np.zeros(n)
    for i in range(n):
        Rgl_in = PR.gl_input(L.d[i], L.lmd[i])
        Rgl[i] = PR.Glazing_Resistance(Rgl_in)
    
    q_a = np.zeros(2 * n + 1)
    q_a[0] = 0.
    for i in range(1, n + 1):   
        q_a[2 * (i - 1) + 1] = L.ia[i - 1] / 2.
        q_a[2 * i] = L.ia[i - 1] / 2.
        
    r_t = np.zeros(2 * n + 2)
    told = np.zeros(2 * n + 2)
    r_R = np.zeros(2 * n + 1)
    Rold = np.zeros(2 * n + 1)

    #温度の初期値
    for i in range(0, 2 * n + 2):
        r_t[i] = N.te +(N.ti - N.te) * i / (2 * n + 1)
        told[i] = r_t[i]

    while True:
        #前回の温度
        for i in range(2 * n + 2):
            told[i] = r_t[i]
    
        #前回の熱抵抗
        for i in range(2 * n + 1):
            Rold[i] = r_R[i]

        #面材間の中空層の熱抵抗
        Rs = np.zeros(n - 1)
        for i in range(n - 1):
            Rs_in = CCR.gap_input(r_t[2 * (i + 1)], r_t[2 * (i + 1) + 1], L.epb[i], L.epf[i + 1], \
                                                       M.d[i], M.gas1[i], M.gas2[i], M.gas1ratio[i], M.gasdir[i])
            Rs[i] = CCR.Gap_Resistance(Rs_in)

        #表面熱伝達抵抗（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）
        Rsur_in = [SR.shc_input(r_t[1], N.te, L.epf[0], 2 * N.season + 1), 
                   SR.shc_input(r_t[2 * n], N.ti, L.epb[n - 1], 2 * N.season)]
        
        Rsur = np.zeros(len(Rsur_in))
        for i in range(len(Rsur_in)):
            Rsur[i] = SR.Surface_Resistance(Rsur_in[i])

        #熱抵抗の配列
        r_R[0] = Rsur[0]
        for i in range(n):
            r_R[2 * i + 1] = Rgl[i]
        for i in range(n - 1):
            r_R[2 * (i + 1)] = Rs[i]
        r_R[2 * n] = Rsur[1]

        #行列式の生成
        ca = np.zeros((2 * n, 2 * n))
        cb = np.zeros((2 * n))
        cx = np.zeros((2 * n))

        for i in range(2 * n + 1):
            if r_R[i] == 0.:
                r_R[i] = 10. ** (-10.)
        for i in range(2 * n):
            ca[i][i] = 1/ r_R[i] + 1/ r_R[i + 1]
            if i == 0:
                cb[i] = q_a[i + 1] + r_t[i] / r_R[i]
            elif i == 2 * n - 1:
                cb[i] = q_a[i + 1] + r_t[i + 2] / r_R[i + 1]
            else:
                cb[i] = q_a[i + 1]

        for i in range(2 * n - 1):
            ca[i + 1][i] = - 1/ r_R[i + 1]
            ca[i][i + 1] = - 1/ r_R[i + 1]
        #修正コレスキー分解
        cx = modified_Cholesky_decomposition(ca, cb)

        #表面温度の取得
        for i in range(0, 2 * n):
            r_t[i + 1] = cx[i]

        #温度の収束判定
        tcdt = 1
        cdt =  np.zeros(2 * n + 2)
        for i in range(2 * n + 2):
            if abs(r_t[i] - told[i]) < 10 **(-10):
                cdt[i] = 1
            else:
                cdt[i] = 0
            tcdt = tcdt * cdt[i]

        #熱抵抗の収束判定
        tcdR = 1
        cdR =  np.zeros(2 * n + 1)
        for i in range(0, 2 * n + 1):
            if abs(r_R[i] - Rold[i]) < 10 **(-10):
                cdR[i] = 1
            else:
                cdR[i] = 0
            tcdR = tcdR * cdR[i]

        tcd = tcdt + tcdR
        if tcd == 2:
            break

    return r_t, r_R

#### 修正コレスキー分解

修正コレスキー分解の解法を以下に示す。  

$$
\begin{eqnarray}
 &\displaystyle \boldsymbol{Ax} = \boldsymbol{b}& \qquad\qquad\text{(5)} \nonumber\\
\end{eqnarray}
$$

ここで、

$ 
\boldsymbol{A}=
\begin{pmatrix} 
 a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
 a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
 \vdots  & \vdots  & \ddots & \vdots  \\
 a_{n,1} & a_{n,2} & \cdots & a_{n,n} \\
\end{pmatrix}
$
、
$
\boldsymbol{x}=
\begin{pmatrix} 
 x_{1} \\
 x_{2} \\
 \vdots \\
 x_{n} \\
\end{pmatrix}
$
、
$
\boldsymbol{b}=
\begin{pmatrix} 
 b_{1} \\
 b_{2} \\
 \vdots \\
 b_{n} \\
\end{pmatrix}
$

ただし、
$
\begin{eqnarray}
 \displaystyle a_{i,j} = a_{j,i} \nonumber\\
\end{eqnarray}
$

行列$\boldsymbol{A}$を以下のように分解する。

$$
\begin{eqnarray}
 &\displaystyle \boldsymbol{A} = \boldsymbol{LDL}^T& \qquad\qquad\text{(6)} \nonumber\\
\end{eqnarray}
$$

ここで、

$ 
\boldsymbol{L}=
\begin{pmatrix} 
 1 &&&& \\
 l_{2,1} & 1 &&O& \\
 l_{3,1} & l_{3,2} & 1 && \\
 \vdots  & \vdots  & \vdots  & \ddots & \\
 l_{n,1} & l_{n,2} & l_{n,3} & \cdots & 1 \\
\end{pmatrix}
$
、
$
\boldsymbol{D}=
\begin{pmatrix} 
 d_{1,1} &&&& \\
 & d_{2,2} &&O& \\
 && \ddots && \\
 &O&& \ddots & \\
 &&&& d_{n,n} \\
\end{pmatrix}
$
、
$
\boldsymbol{L}^T=
\begin{pmatrix} 
 1 & l_{1,2} & l_{1,3} & \cdots & l_{1,n} \\
 & 1 & l_{2,2} & \cdots & l_{2,n} \\
 && 1 & \cdots & l_{3,n} \\
 &O&& \ddots &\vdots \\
 &&&& 1 \\
\end{pmatrix}
$  

ただし、
$
\begin{eqnarray}
 \displaystyle l_{i,j} = l_{j,i} \nonumber\\
\end{eqnarray}
$

この関係を整理すると、以下のようになる。

$$
\begin{eqnarray}
 &\displaystyle d_{1,1} = a_{1,1}& \qquad\qquad\text{(7)} \nonumber\\
 \nonumber \\
 &\displaystyle l_{k,i} = \frac{a_{k,i} - \sum_{j=1}^{i-1} l_{k,j} \cdot l_{i,j} \cdot d_{j,j}}{d_{i,i}}& \qquad\qquad\text{(8)} \nonumber\\
 \nonumber \\
 &\displaystyle d_{k,k} = a_{k,k} - \sum_{i=1}^{k-1} l_{k,i}^2 \cdot d_{i,i}& \qquad\qquad\text{(9)} \nonumber\\
\end{eqnarray}
$$

式(6)より、 

$$
\begin{eqnarray}
 &\displaystyle \boldsymbol{LDL}^T \boldsymbol{x} = \boldsymbol{b}& \qquad\qquad\text{(10)} \nonumber\\
\end{eqnarray}
$$

$
\begin{eqnarray}
 \displaystyle \boldsymbol{DL}^T \boldsymbol{x} = \boldsymbol{y}\nonumber\\
\end{eqnarray}
$
とすると、

$$
\begin{eqnarray}
 &\displaystyle \boldsymbol{Ly} = \boldsymbol{b}& \qquad\qquad\text{(11)} \nonumber\\
\end{eqnarray}
$$

ここで、  

$ 
\boldsymbol{y}=
\begin{pmatrix} 
 y_{1} \\
 y_{2} \\
 \vdots \\
 y_{n} \\
\end{pmatrix}
$

この関係を整理すると、以下のようになる。  

$$
\begin{eqnarray}
 &\displaystyle y_i = b_i \quad (i=1)& \qquad\qquad\text{(12)} \nonumber\\
 \nonumber \\
 &\displaystyle y_i = b_i - \sum_{j=1}^{n-1} l_{i,j} \cdot y_j \quad (2\leq i \leq n)& \qquad\qquad\text{(13)} \nonumber\\
 \nonumber \\
 &\displaystyle x_i = \frac{y_i}{d_{i,i}} \quad (i=n)& \qquad\qquad\text{(14)} \nonumber\\
 \nonumber \\
 &\displaystyle x_i = \frac{y_i - d_{i,i} \cdot \sum_{j=i+1}^{n} l_{j,i} \cdot x_j}{d_{i,i}} \quad (1\leq i \leq n-1)& \qquad\qquad\text{(15)} \nonumber\\
\end{eqnarray}
$$

In [4]:
#修正コレスキー分解
def modified_Cholesky_decomposition(a, b):

    n = len(a)
    d = np.zeros(n)
    l = np.zeros((n,n))
    x = np.zeros(n)
    y = np.zeros(n)

    d[0] = a[0][0]

    for i in range(n):
        l[i][i] = 1.

    for k in range(1,n):
        s = 0.
        t = 0.
        for i in range(k):
            for j in range(i):
                s -= l[k][j] * l[i][j] * d[j]
            l[k][i] = (a[k][i] + s) / d[i]

        for i in range(k):
            t -= l[k][i] ** 2. * d[i]

        d[k] = a[k][k] + t

    y[0] = b[0]
    for i in range(1,n):
        u = 0.
        for j in range(n-1):
            u -= l[i][j] * y[j]

        y[i] = b[i] + u

    x[n-1] = y[n-1] / d[n-1]
    for i in range(n-2,-1,-1):
        v = 0.
        for j in range(n-1,i,-1):
            v -= l[j][i] * x[j]

        x[i] = y[i] / d[i] + v

    return x

#### Example

In [5]:
if __name__ == '__main__':

    #グレージングの厚さ
    gl_d = [[0.003, 0.006], [0.003], [0.003]]

    #グレージングの熱伝導率
    gl_lmd = [[1., 0.5], [1.], [1.]]

    #各層の吸収日射量
    gl_ia = [9.55935027, 6.8267886, 4.76774099]

    #各層の正面側の放射率
    gl_epf = [0.837, 0.837, 0.837]

    #各層の背面側の放射率
    gl_epb = [0.837, 0.837, 0.837]

    #中空層の厚さ
    gp_d = [0.012, 0.012]

    #中空層の気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
    gp_gas1 = [0, 0]

    #中空層の気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
    gp_gas2 = [0, 0]

    # 中空層の気体1の容積割合（0.0～1.0）
    gp_gas1r = [1., 1.]

    # 中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）
    gp_dir = [0, 0]

    season = 0 #夏期：0、冬期：1
    if season == 0:
        te = 30.  #外気温
        ti = 25.  #室温
    elif season == 1:
        te = 0.   #外気温
        ti = 20.  #室温

    glp_in = glp_input(gl_d, gl_lmd, gl_ia, gl_epf, gl_epb)
    gpp_in = gpp_input(gp_d, gp_gas1, gp_gas2, gp_gas1r, gp_dir)
    wp_in = wp_input(season, te, ti)

    hbr = Heat_balance(glp_in, gpp_in, wp_in)

    tglz = hbr[0][0 : 2 * (len(gl_d) + 1)]
    Rglz = hbr[1][0 : 2 * len(gl_d) + 1]

    print ('tglz', tglz)
    print ('Rglz', Rglz)

tglz [30.         30.26096248 30.24131152 29.33911229 29.31060289 27.37569226
 27.32979106 25.        ]
Rglz [0.07521376 0.015      0.14815071 0.003      0.14980114 0.003
 0.1317437 ]
